# 以固定间隔枚举目标空间采样

In [30]:
from pathlib import Path
import sys
import gymnasium as gym
import panda_gym
import numpy as np
import pandas as pd
from gymnasium.envs.registration import register

PROJECT_ROOT_DIR = Path().absolute().parent
PROJECT_ROOT_DIR

if str(PROJECT_ROOT_DIR.absolute()) not in sys.path:
    sys.path.append(str(PROJECT_ROOT_DIR.absolute()))

from rollout.rollout_by_pid import rollout_by_goal_with_pid
from my_reach_env import MyPandaReachEnv


In [ ]:
goal_range = 0.3
distance_threshold = 0.01

In [31]:
register(
    id="my-reach",
    entry_point=f"my_reach_env:MyPandaReachEnv",
    kwargs={"reward_type": "sparse", "control_type": "ee", "goal_range": goal_range, "distance_threshold": distance_threshold},
    max_episode_steps=50,
)

/home/ucav/anaconda3/envs/panda_gym/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment my-reach already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [32]:
env = gym.make("my-reach")
env.observation_space

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


Dict('achieved_goal': Box(-10.0, 10.0, (3,), float32), 'desired_goal': Box(-10.0, 10.0, (3,), float32), 'observation': Box(-10.0, 10.0, (6,), float32))

In [33]:
env.task.goal_range_low, env.task.goal_range_high

(array([-0.05, -0.05,  0.  ]), array([0.05, 0.05, 0.1 ]))

In [34]:
env.task.distance_threshold

0.01

In [36]:
success_cnt = 0
total_df = None
goal_range_int = int(goal_range * 100)
speed=2
x_low, x_high = -int(goal_range_int/2), int(goal_range_int/2)
y_low, y_high = -int(goal_range_int/2), int(goal_range_int/2)
z_low, z_high = 0, int(goal_range_int)
for x in range(x_low, x_high+1, 2):
    for y in range(y_low, y_high+1, 2):
        for z in range(z_low, z_high+1, 2):
            terminated, truncated, traj = rollout_by_goal_with_pid(env=env, goal=np.array([x/100., y/100., z/100.]), speed=speed)
            if terminated:
                if total_df is None:
                    total_df = pd.DataFrame(traj)
                else:
                    total_df = pd.concat([total_df, pd.DataFrame(traj)])
                success_cnt += 1

success_cnt

 success. goal: (-0.15, -0.15, 0.0), steps: 43 
 success. goal: (-0.15, -0.15, 0.02), steps: 41 
 success. goal: (-0.15, -0.15, 0.04), steps: 41 
 success. goal: (-0.15, -0.15, 0.06), steps: 41 
 success. goal: (-0.15, -0.15, 0.08), steps: 40 
 success. goal: (-0.15, -0.15, 0.1), steps: 40 
 success. goal: (-0.15, -0.15, 0.12), steps: 40 
 success. goal: (-0.15, -0.15, 0.14), steps: 39 
 success. goal: (-0.15, -0.15, 0.16), steps: 39 
 success. goal: (-0.15, -0.15, 0.18), steps: 39 
 success. goal: (-0.15, -0.15, 0.2), steps: 39 
 success. goal: (-0.15, -0.15, 0.22), steps: 39 
 success. goal: (-0.15, -0.15, 0.24), steps: 39 
 success. goal: (-0.15, -0.15, 0.26), steps: 39 
 success. goal: (-0.15, -0.15, 0.28), steps: 39 
 success. goal: (-0.15, -0.15, 0.3), steps: 39 
 success. goal: (-0.15, -0.13, 0.0), steps: 42 
 success. goal: (-0.15, -0.13, 0.02), steps: 41 
 success. goal: (-0.15, -0.13, 0.04), steps: 41 
 success. goal: (-0.15, -0.13, 0.06), steps: 40 
 success. goal: (-0.15, -

3782

In [37]:
total_df

,s_x,s_y,s_z,s_v_x,s_v_y,s_v_z,s_g_x,s_g_y,s_g_z,a_x,a_y,a_z
0,0.038440,-2.194472e-12,0.197400,0.000000,-0.000000,0.000000,-0.15,-0.15,0.00,-0.376879,-0.300000,-0.394800
1,0.027911,-1.200392e-02,0.181376,-0.167180,-0.159938,-0.212242,-0.15,-0.15,0.00,-0.355823,-0.275992,-0.362753
2,0.016768,-2.379170e-02,0.166082,-0.117814,-0.111183,-0.124885,-0.15,-0.15,0.00,-0.333537,-0.252417,-0.332164
3,0.006184,-3.454186e-02,0.152202,-0.121417,-0.105164,-0.124790,-0.15,-0.15,0.00,-0.312368,-0.230916,-0.304403
4,-0.004057,-4.443558e-02,0.139541,-0.115374,-0.093868,-0.120351,-0.15,-0.15,0.00,-0.291886,-0.211129,-0.279082
...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.137838,9.653389e-03,0.160080,0.003811,0.000146,-0.000338,0.15,0.01,0.16,0.024324,0.000693,-0.000161
46,0.138313,9.671042e-03,0.160041,0.003643,0.000135,-0.000301,0.15,0.01,0.16,0.023374,0.000658,-0.000083
47,0.138767,9.687416e-03,0.160007,0.003484,0.000126,-0.000267,0.15,0.01,0.16,0.022466,0.000625,-0.000014
48,0.139202,9.702616e-03,0.159976,0.003333,0.000117,-0.000237,0.15,0.01,0.16,0.021597,0.000595,0.000048


In [38]:
total_df.to_csv(f"myreach_pid_speed_{speed:.1f}.csv", index=False)